In [1]:
import pandas as pd
import numpy as np
import requests
from pathlib import Path

In [2]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [5]:
DATA = Path('data')
!ls {DATA}

ari_2018.csv	  medical_college_list.csv  pneumonia_2018.csv
doctors.csv	  nhrr			    pop_by_age_2017.csv
govt_doctors.csv  nurses.csv		    population.csv


In [6]:
college = pd.read_csv(DATA/'medical_college_list.csv')

In [7]:
r = requests.get('https://api.rootnet.in/covid19-in/stats/hospitals')
r.status_code

200

In [8]:
res = r.json()

In [11]:
hospital = pd.DataFrame(res['data']['regional'])

In [12]:
hospital.head()

,state,ruralHospitals,ruralBeds,urbanHospitals,urbanBeds,totalHospitals,totalBeds,asOn
0,Andhra Pradesh,193,6480,65,16658,258,23138,2017-01-01T00:00:00.000Z
1,Arunachal Pradesh,208,2136,10,268,218,2404,2017-12-31T00:00:00.000Z
2,Assam,1176,10944,50,6198,1226,17142,2017-12-31T00:00:00.000Z
3,Bihar,930,6083,103,5936,1033,12019,2016-12-31T00:00:00.000Z
4,Chhattisgarh,169,5070,45,4342,214,9412,2016-01-01T00:00:00.000Z


In [13]:
hospital.columns = ['state', 'rural_hospital', 'rural_bed', 'urban_hospital', 'urban_bed', 'hospital', 'bed', 'date']
hospital.head()

,state,rural_hospital,rural_bed,urban_hospital,urban_bed,hospital,bed,date
0,Andhra Pradesh,193,6480,65,16658,258,23138,2017-01-01T00:00:00.000Z
1,Arunachal Pradesh,208,2136,10,268,218,2404,2017-12-31T00:00:00.000Z
2,Assam,1176,10944,50,6198,1226,17142,2017-12-31T00:00:00.000Z
3,Bihar,930,6083,103,5936,1033,12019,2016-12-31T00:00:00.000Z
4,Chhattisgarh,169,5070,45,4342,214,9412,2016-01-01T00:00:00.000Z


In [14]:
college.columns = ['state', 'name', 'city', 'type', 'capacity', 'bed']
college.head()

,state,name,city,type,capacity,bed
0,Andaman & Nicobar Islands,Andaman & Nicobar Islands Insitute of Medical ...,Port Blair,Govt.,100.0,460
1,Andhra Pradesh,ACSR Government Medical College Nellore,Nellore,Govt.,150.0,750
2,Andhra Pradesh,"All India Institute of Medical Sciences, Manga...",Vijaywada,Govt.,50.0,NaN
3,Andhra Pradesh,Alluri Sitaram Raju Academy of Medical Science...,Eluru,Trust,150.0,1070
4,Andhra Pradesh,"Andhra Medical College, Visakhapatnam",Visakhapatnam,Govt.,200.0,2017


In [15]:
college['bed'] = (college['bed']
                     .replace(to_replace={'Na': np.nan})
                     .fillna(0)
                     .astype('int'))

In [16]:
college_capacity = (college
                      .groupby('state')
                      .agg({
                          'bed': 'sum',
                          'name': 'size'
                      }))

In [17]:
hospital_capacity = hospital[['state', 'bed', 'hospital']]

In [18]:
df = pd.merge(hospital_capacity, college_capacity, on='state', how='left')

In [19]:
df['bed_p'] = (df['bed_y'] / df['bed_x'] * 100)
df['hospital_p'] = (df['name'] / df['hospital'] * 100)

In [20]:
df

,state,bed_x,hospital,bed_y,name,bed_p,hospital_p
0,Andhra Pradesh,23138,258,20410.0,31.0,88.209871,12.015504
1,Arunachal Pradesh,2404,218,0.0,1.0,0.000000,0.458716
2,Assam,17142,1226,5653.0,6.0,32.977482,0.489396
3,Bihar,12019,1033,7167.0,15.0,59.630585,1.452081
4,Chhattisgarh,9412,214,5139.0,10.0,54.600510,4.672897
5,Goa,3013,42,1160.0,1.0,38.499834,2.380952
6,Gujarat,32280,486,19703.0,29.0,61.037794,5.967078
7,Haryana,11240,668,4223.0,12.0,37.571174,1.796407
8,Himachal Pradesh,12399,801,2028.0,7.0,16.356158,0.873908
9,Jammu & Kashmir,11651,132,5595.0,8.0,48.021629,6.060606


In [23]:
from geopy.geocoders import GoogleV3, Nominatim

In [1]:
G = GoogleV3(api_key='YOUR_API_KEY', domain='maps.google.co.in')

NameError: name 'GoogleV3' is not defined

In [128]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(G.geocode, min_delay_seconds=1)

In [129]:
college['location'] = college['name'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Andaman & Nicobar Islands Insitute of Medical Sciences, \nPort Blair',), **{}).
Traceback (most recent call last):
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/site-packages/geopy/geocoders/base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1360, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1320, in do_open
    r = h.getr

RateLimiter caught an error, retrying (0/2 tries). Called with (*('All India Institute of Medical Sciences  Patna, Bihar',), **{}).
Traceback (most recent call last):
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 966

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Chandulal Chandrakar Memorial Medical College, Durg',), **{}).
Traceback (most recent call last):
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/site-packages/geopy/geocoders/base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1360, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1320, in do_open
    r = h.getresponse()
  File 

RateLimiter caught an error, retrying (0/2 tries). Called with (*('GMERS Medical College, Dharpur Patan',), **{}).
Traceback (most recent call last):
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 966, in send
    sel

RateLimiter caught an error, retrying (0/2 tries). Called with (*('SBKS Medical Instt. & Research Centre, Vadodra',), **{}).
Traceback (most recent call last):
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/site-packages/geopy/geocoders/base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1360, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1320, in do_open
    r = h.getresponse()
  File "/hom

...

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Government Sivgangai Medical College, Sivaganga',), **{}).
Traceback (most recent call last):
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 966, in s

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Madras Medical College, Chennai',), **{}).
Traceback (most recent call last):
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 966, in send
    self.con

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Dr. VRK Womens Medical College, Aziznagar',), **{}).
Traceback (most recent call last):
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 966, in send
  

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Krishna Mohan Medical College & Hospital, Mathura.',), **{}).
Traceback (most recent call last):
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 966, i

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Midnapore Medical College, Midnapore',), **{}).
Traceback (most recent call last):
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/home/srm/anaconda3/envs/geohack/lib/python3.7/http/client.py", line 966, in send
    sel

In [135]:
college.location

0      (Old Ayush Building, Near DHS Annexe Building,...
1      (1483, 25-2-1483, Podalakur Rd, Pragati Nagar,...
2      (Mangalagiri, Andhra Pradesh 522503, India, (1...
3      (Malkapuram, West Godavari NH-5, Vijayawada, V...
4      (Medical College Road Opp Collector Office Mah...
5      (Murukambattu, Andhra Pradesh 517127, India, (...
6      (Chinna Avutapalle,, Peda Avutapalli, Andhra P...
7      (Kadapa, Andhra Pradesh 516003, India, (14.483...
8      (6-25 Marikavalasa, Maridi valley, Village, Sr...
9      (Rushikonda, Visakhapatnam, Andhra Pradesh 530...
10     (Rahamat Nagar, Anantapur, Andhra Pradesh 5150...
11     (NH 16 Service Rd, beside Dr NTR University of...
12     (Gems Medical College Road, Aditya Educational...
13     (NH 16, Lakshmi Puram, Rajahmundry, Andhra Pra...
14     (Kanna Vari Thota, Guntur, Andhra Pradesh 5220...
15     (Katuri Nagar, Guntur, Andhra Pradesh 500019, ...
16     (NH-216 Chaitanya Health City, Amalapuram, And...
17     (Bhudawarapet, Bharath P

In [140]:
college['point'] = college['location'].apply(lambda loc: tuple(loc.point) if loc else None)
college['address'] = college['location'].apply(lambda loc: loc.address if loc else None)
college['lat'] = college['location'].apply(lambda loc: loc.latitude if loc else None)
college['lng'] = college['location'].apply(lambda loc: loc.longitude if loc else None)

In [143]:
college.drop('location', axis=1, inplace=True)

In [145]:
college.to_csv(DATA/'nhrr'/'2020-03-23_college.csv')

In [146]:
df = pd.read_csv(DATA/'nhrr'/'temp.csv')

In [149]:
df.drop(['point', 'address', 'lat', 'lng'], axis=1, inplace=True)

In [ ]:
def geocoder(x):
    G

In [174]:
df['fullname'] = df.name + ',' + df.city + ',' + df.state

In [175]:
df

,id,state,name,city,type,capacity,bed,fullname
0,125,Jammu & Kashmir,Government Medical College,Anantnag,Govt.,NaN,0,"Government Medical College,Anantnag,Jammu & Ka..."
1,127,Jammu & Kashmir,Government Medical College,Baramulla,Govt.,NaN,0,"Government Medical College,Baramulla,Jammu & K..."
2,128,Jammu & Kashmir,Government Medical College,Jammu,Govt.,150.0,2295,"Government Medical College,Jammu,Jammu & Kashmir"
3,129,Jammu & Kashmir,Government Medical College,Kathua,Govt.,NaN,0,"Government Medical College,Kathua,Jammu & Kashmir"
4,234,Madhya Pradesh,Government Medical College,Chhindwara,Govt.,NaN,0,"Government Medical College,Chhindwara,Madhya P..."
5,235,Madhya Pradesh,Government Medical College,Datia,Govt.,100.0,0,"Government Medical College,Datia,Madhya Pradesh"
6,236,Madhya Pradesh,Government Medical College,Khandwa,Govt.,100.0,0,"Government Medical College,Khandwa,Madhya Pradesh"
7,237,Madhya Pradesh,Government Medical College,Ratlam,Govt,150.0,0,"Government Medical College,Ratlam,Madhya Pradesh"
8,238,Madhya Pradesh,Government Medical College,Sahadol,Govt.,NaN,0,"Government Medical College,Sahadol,Madhya Pradesh"
9,239,Madhya Pradesh,Government Medical College,Shivpuri,Govt.,NaN,0,"Government Medical College,Shivpuri,Madhya Pra..."


In [176]:
loc = G.geocode(query='Government Medical College,Nalgonda,Telangana')

In [177]:
loc

Location(Rahamath Nagar, Ramgiri, Rahamath Nagar, Ramgiri, Nalgonda, Telangana 508003, भारत, (17.0558638, 79.27788389999999, 0.0))

In [210]:
temp['location'] = temp['fullloc'].apply(geocode)

/home/srm/anaconda3/envs/geohack/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [211]:
temp

,id,state,name,city,type,capacity,bed,point,address,lat,lng,fullloc,location
84,84,Gujarat,"GMERS Medical College, Gandhinagar",Gandhinagar,Govt-Society,150.0,650,None,None,NaN,NaN,"GMERS Medical College, Gandhinagar,Gandhinagar...",None
126,126,Jammu & Kashmir,Government Medical College & Associated Hospital,Rajouri,Govt.,NaN,0,None,None,NaN,NaN,Government Medical College & Associated Hospit...,"(Kheora, Rajouri, 185131, (33.3878401, 74.3156..."
176,176,Karnataka,S S Institute of Medical Sciences& Research Ce...,Davangere,Trust,150.0,930,None,None,NaN,NaN,S S Institute of Medical Sciences& Research Ce...,None
232,232,Madhya Pradesh,"Gajra Raja Medical College, Gwalior",Gwalior,Govt.,150.0,1320,None,None,NaN,NaN,"Gajra Raja Medical College, Gwalior,Gwalior,Ma...",None
302,302,Maharashtra,"Topiwala National Medical College, Mumbai",Mumbai,Govt.,120.0,0,None,None,NaN,NaN,"Topiwala National Medical College, Mumbai,Mumb...",None
475,475,Uttar Pradesh,Manyavar Kanshi Ram Ji Government Allopathic M...,Kanpur,Govt.,100.0,300,None,None,NaN,NaN,Manyavar Kanshi Ram Ji Government Allopathic M...,"(Kalpi Road, Orai, Uttar Pradesh 285001, India..."


In [212]:
temp['point'] = temp['location'].apply(lambda loc: tuple(loc.point) if loc else None)
temp['address'] = temp['location'].apply(lambda loc: loc.address if loc else None)
temp['lat'] = temp['location'].apply(lambda loc: loc.latitude if loc else None)
temp['lng'] = temp['location'].apply(lambda loc: loc.longitude if loc else None)

/home/srm/anaconda3/envs/geohack/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/srm/anaconda3/envs/geohack/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/srm/anaconda3/envs/geohack/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [213]:
temp

,id,state,name,city,type,capacity,bed,point,address,lat,lng,fullloc,location
84,84,Gujarat,"GMERS Medical College, Gandhinagar",Gandhinagar,Govt-Society,150.0,650,None,None,NaN,NaN,"GMERS Medical College, Gandhinagar,Gandhinagar...",None
126,126,Jammu & Kashmir,Government Medical College & Associated Hospital,Rajouri,Govt.,NaN,0,"(33.3878401, 74.3156175, 0.0)","Kheora, Rajouri, 185131",33.387840,74.315618,Government Medical College & Associated Hospit...,"(Kheora, Rajouri, 185131, (33.3878401, 74.3156..."
176,176,Karnataka,S S Institute of Medical Sciences& Research Ce...,Davangere,Trust,150.0,930,None,None,NaN,NaN,S S Institute of Medical Sciences& Research Ce...,None
232,232,Madhya Pradesh,"Gajra Raja Medical College, Gwalior",Gwalior,Govt.,150.0,1320,None,None,NaN,NaN,"Gajra Raja Medical College, Gwalior,Gwalior,Ma...",None
302,302,Maharashtra,"Topiwala National Medical College, Mumbai",Mumbai,Govt.,120.0,0,None,None,NaN,NaN,"Topiwala National Medical College, Mumbai,Mumb...",None
475,475,Uttar Pradesh,Manyavar Kanshi Ram Ji Government Allopathic M...,Kanpur,Govt.,100.0,300,"(26.0001572, 79.4741813, 0.0)","Kalpi Road, Orai, Uttar Pradesh 285001, India",26.000157,79.474181,Manyavar Kanshi Ram Ji Government Allopathic M...,"(Kalpi Road, Orai, Uttar Pradesh 285001, India..."


In [215]:
temp.drop(['location', 'fullloc'], axis=1, inplace=True)

/home/srm/anaconda3/envs/geohack/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [217]:
df = pd.concat([df, temp]).drop_duplicates(subset='id', keep='last').sort_values('id')

In [218]:
temp = df[df.point.isnull()]

In [219]:
temp

,id,state,name,city,type,capacity,bed,point,address,lat,lng
84,84,Gujarat,"GMERS Medical College, Gandhinagar",Gandhinagar,Govt-Society,150.0,650,None,None,NaN,NaN
176,176,Karnataka,S S Institute of Medical Sciences& Research Ce...,Davangere,Trust,150.0,930,None,None,NaN,NaN
232,232,Madhya Pradesh,"Gajra Raja Medical College, Gwalior",Gwalior,Govt.,150.0,1320,None,None,NaN,NaN
302,302,Maharashtra,"Topiwala National Medical College, Mumbai",Mumbai,Govt.,120.0,0,None,None,NaN,NaN


In [222]:
df.to_csv(DATA/'nhrr'/'2020-03-23_medical_college_india.csv', index=False)